[爬蟲案例分享]
- 難點：

    1.資料量大，但每個查詢最多150個分頁，因此需要將查詢的範圍限縮。
    解決方式是在查詢時組合地區與職務類型，進行細緻的查詢。

    2.雖然幾乎沒有設定反爬蟲的機制，但每個 request 送的速度太快仍然會有失敗的狀況發生
    用while迴圈，當request成功是繼續前往下一頁爬蟲，request成功但資料小於20筆，查詢下一個組合，request等於20，表示還有資料需要繼續抓下一個分頁的資料。

    3.因為資料量大，建議按照組別存一次資料，這樣遇到錯誤時才不用重新爬資料。

    
- 因為查詢職缺最多有20\*150=3000筆資料，所以要將查詢的的條件透過條件進行篩選、排列組合，藉此撈出完整的資料
- 由公司資訊：地址，聯絡電話、職務
- 縣市和職務的分類有個資的代號，但中間有經過變(如縣市合併)更並不是單純的序列，因此要花時間找一下
- 可以看各個縣市的職缺數量與職務類型，進一步分析需要什麼技能
- 你要在哪裡，需要什麼技能、待遇如何
- 可以去串各縣市的人口數、教育程度等資訊
- 縣市的產業結構
- 可行的分析項目

# 載入套件

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
import gc
from IPython.display import clear_output
from tomorrow import threads
import pickle
# import grequests

In [4]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
         'Referer': 'https://www.104.com.tw/',
         'Host': 'www.104.com.tw',
         'DNT':'1',
         'Cookie':'GoogleBot'}

# 職缺清單
- 透過地區與職務分類來請求職缺資料

In [3]:
# 地區代碼
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url)
df1 = []
for i in resp.json()[0]['n']:
    ndf = pd.DataFrame(i['n'])
    ndf['city'] = i['des']
    df1.append(ndf)
df1=pd.concat(df1, ignore_index=True)
df1 = df1.loc[:,['city','des','no']]
df1 = df1.sort_values('no')
df1.head()

,city,des,no
0,台北市,台北市中正區,6001001001
1,台北市,台北市大同區,6001001002
2,台北市,台北市中山區,6001001003
3,台北市,台北市松山區,6001001004
4,台北市,台北市大安區,6001001005


In [4]:
url= 'https://static.104.com.tw/category-tool/json/JobCat.json'
resp = requests.get(url)
df2 = []
for i in resp.json():
    for j in i['n']:
        ndf = pd.DataFrame(j['n'])
        ndf.columns = ['des3','no3']
        ndf['des2'] = j['des']# 職務中分類
        ndf['no2'] = j['no']
        ndf['des1'] = i['des']# 職務大分類
        ndf['no1'] = i['no']
        df2.append(ndf)
df2 = pd.concat(df2, ignore_index=True)
df2 = df2.sort_values('no3')
print(df2.shape)
df2.head()

(466, 6)


,des3,no3,des2,no2,des1,no1
1,經營管理主管,2001001001,經營╱幕僚類人員,2001001000,經營╱人資類,2001000000
0,儲備幹部,2001001002,經營╱幕僚類人員,2001001000,經營╱人資類,2001000000
2,主管特別助理,2001001003,經營╱幕僚類人員,2001001000,經營╱人資類,2001000000
3,人力資源主管,2001002001,人力資源類人員,2001002000,經營╱人資類,2001000000
6,人力資源人員,2001002002,人力資源類人員,2001002000,經營╱人資類,2001000000


In [5]:
# 捨棄最細的職務分類，減少重複爬相同職缺的情況
df2 = df2.drop_duplicates(['no2']).reset_index(drop=True)
print(df2.shape)
df2.head()

(44, 6)


,des3,no3,des2,no2,des1,no1
0,經營管理主管,2001001001,經營╱幕僚類人員,2001001000,經營╱人資類,2001000000
1,人力資源主管,2001002001,人力資源類人員,2001002000,經營╱人資類,2001000000
2,行政主管,2002001001,行政╱總務類人員,2002001000,行政╱總務╱法務類,2002000000
3,法務╱智財主管,2002002001,法務╱智財類人員,2002002000,行政╱總務╱法務類,2002000000
4,財務或會計主管,2003001001,財務╱會計╱稅務類,2003001000,財會╱金融專業類,2003000000


In [1]:
# # executed in 3h 12m 58s, finished 19:38:48 2020-09-20
# jobs = []
# for areades, areacode in zip(df1['des'],df1['no']):
#     for no2, des2 in zip(df2['no2'], df2['des2']):
#         print(areades, ' | ', no2, ' - ', des2)
#         page = 1
#         while page <100:
#             try:
#                 # ro=1為全職
#                 # isnew=31表示近31天有更新(表示職缺有效)
#                 url = 'https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&jobcat={}&jobcatExpansionType=1&area={}&order=11&asc=0&page={}&mode=l&jobsource=2018indexpoc'.format(no2, areacode, page)
#                 print(url)
#                 resp = requests.get(url, headers=headers, timeout=5)
#                 ndf = pd.DataFrame(resp.json()['data']['list'])
#                 print(ndf.shape)
#                 jobs.append(ndf)
#                 if ndf.shape[0]<30:
#                     break
#                 else:
#                     page += 1
#             except:
#                 print('Retry')
        
# jobs = pd.concat(jobs, ignore_index=True)
# jobs.to_pickle('./data/data.pkl')
# print('===================================  Save Data  ===================================')

In [8]:
# 不限全職/兼職：446332
# 限全職:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352351 entries, 0 to 352350
Data columns (total 34 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   jobType         352351 non-null  object
 1   jobNo           352351 non-null  object
 2   jobName         352351 non-null  object
 3   jobNameSnippet  352351 non-null  object
 4   jobRole         352351 non-null  object
 5   jobRo           352351 non-null  object
 6   jobAddrNoDesc   352351 non-null  object
 7   jobAddress      352351 non-null  object
 8   description     352351 non-null  object
 9   optionEdu       352351 non-null  object
 10  period          352351 non-null  object
 11  periodDesc      352351 non-null  object
 12  applyCnt        352351 non-null  object
 13  applyDesc       352351 non-null  object
 14  custNo          352351 non-null  object
 15  custName        352351 non-null  object
 16  coIndustry      352351 non-null  object
 17  coIndustryDesc  352351 non-nu

# 職務內容
會擋IP，先用外層的資料分析，後續有時間再來處理代理IP和多線程的問題

## requests

In [3]:
jobs = pd.read_pickle('./data/data.pkl')
jobs = jobs.loc[:,['link']]
jobs['link'] = jobs['link'].apply(lambda x: 'https:' + x['job'].split('?')[0])
jobs['code'] = jobs['link'].apply(lambda x: x.split(r'/')[-1])
jobs['ajax'] = jobs['code'].apply(lambda x: 'https://www.104.com.tw/job/ajax/content/' + x)

print(jobs.shape)
jobs = jobs.drop_duplicates(['link'])
print(jobs.shape)
jobs.head(3)

(352351, 3)
(211610, 3)


,link,code,ajax
0,https://www.104.com.tw/job/71p3w,71p3w,https://www.104.com.tw/job/ajax/content/71p3w
1,https://www.104.com.tw/job/59cwj,59cwj,https://www.104.com.tw/job/ajax/content/59cwj
2,https://www.104.com.tw/job/6o0xa,6o0xa,https://www.104.com.tw/job/ajax/content/6o0xa


In [4]:
@threads(30)
def MultiThread_Crawler(url, code):
    try:
        data = requests.get(url, headers=headers).json()
        with open('data/JobDetail/' + code + '.pickle', 'wb') as handle:
            pickle.dump(data, handle)
    except:
        pass

In [5]:
[MultiThread_Crawler(link, code) for link, code in zip(jobs['ajax'], jobs['code'])]

 ...]

In [6]:
print('Hello')

Hello


## parse

- 評估職務類別要不要保留編號

In [5]:
len(os.listdir('data/JobDetail'))

200830

In [ ]:
df = []
for i, filename in enumerate(os.listdir('data/JobDetail')):
    if i % 100 == 0:
        print(i)
    try:
        data = pd.read_pickle('data/JobDetail/'+ filename)['data']
        ndf = pd.DataFrame([{'jobName': data['header']['jobName'],  # 職稱
                             'appearDate':data['header']['appearDate'],  # 刊登日期
                             'custName':data['header']['custName'],  # 公司名稱
#                              'custUrl':data['header']['custUrl'],  # 公司網址
                             ############ 聯絡方式 ############
                             'hrName':data['contact']['hrName'],  # 聯絡人姓名
                             'email':data['contact']['email'],  # Mail
                             'phone':data['contact']['phone'],  # 電話
#                              'other':data['contact']['other'],  # 其他
                             ############ 條件要求 ############
                             # 接受身份
                             'acceptRole':data['condition']['acceptRole']['role'],
                             'workExp':data['condition']['workExp'],  # 工作經歷
                             'edu':data['condition']['edu'],  # 教育程度
                             # 科系要求
                             'major':data['condition']['major'],
                             'language':data['condition']['language'],  # 語言
                             # 擅長工具
                             'specialty':data['condition']['specialty'],
                             # 工作技能
                             'skill':data['condition']['skill'],
                             'other':data['condition']['other'],  # 其他條件
                             ############ 福利制度 ############
                             # 其他福利
                             'welfare_tag':data['welfare']['tag'],
                             'welfare':data['welfare']['welfare'],  # 福利制度
                             'legalTag':data['welfare']['legalTag'],  # 法定項目
                             ############ 工作內容 ############
                             # 工作內容
                             'jobDetail':data['jobDetail']['jobDescription'],
                             # 職務類別
                             'jobCategory': data['jobDetail']['jobCategory'],
                             'salary':data['jobDetail']['salary'],  # 工作待遇
                             'salaryMin':data['jobDetail']['salaryMin'],  # 最低待遇
                             'salaryMax':data['jobDetail']['salaryMax'],  # 最高待遇
                             'salaryType':data['jobDetail']['salaryType'],# 月薪/日薪/時薪
                             'jobType':data['jobDetail']['jobType'],  # 全職/兼職
                             'workType':data['jobDetail']['workType'],  # 短期/長期工讀標籤
                             # 縣市/區域
                             'addressRegion':data['jobDetail']['addressRegion'],
                             # 地址
                             'addressDetail':data['jobDetail']['addressDetail'],
#                              'industryArea':data['jobDetail']['industryArea'],
#                              'longitude':data['jobDetail']['longitude'],  # 經度
#                              'latitude':data['jobDetail']['latitude'],  # 維度
                             'manageResp':data['jobDetail']['manageResp'],  # 管理職
                             'businessTrip':data['jobDetail']['businessTrip'],# 是否需出差
                             # 班別，日/夜/輪班
                             'workPeriod':data['jobDetail']['workPeriod'],
                             # 周休/排休...
                             'vacationPolicy':data['jobDetail']['vacationPolicy'],
                             # 可上班日
                             'startWorkingDay':data['jobDetail']['startWorkingDay'],
                             'hireType':data['jobDetail']['hireType'],  # 不確定是不是派遣??
#                              'delegatedRecruit':data['jobDetail']['delegatedRecruit'], # ??
#                              'needEmp':data['jobDetail']['needEmp'],  # 需求人數
#                              'switch':data['switch'],  # ??
                             'industry':data['industry']  # 產業
                             }])
        df.append(ndf)
    except:
        print(data)
df = pd.concat(df, ignore_index=True)
df

0
{'header': {'jobName': '業務儲備幹部 (保障前9個月每月5萬元!) C71', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
{'header': {'jobName': '業務儲備幹部 (保障前9個月每月5萬元) C71', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
10500
10600
10700
10800
10900
{'corpImageRight': {'corpImageRight': {'imageUrl': '', 'link': ''}}, 'header': {'corpImageTop': {'imageUrl': '', 'link': ''}, 'jobName': '區域業務專員(彰投)', 'appearDate': '2020/09/14', 'custName': '卡

29200
29300
29400
29500
29600
29700
29800
{'header': {'jobName': '晚班護理人員', 'custName': '陳成福診所'}, 'switch': 'off', 'custNo': '98192538000'}
29900
30000
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
{'header': {'jobName': '不動產儲備經理人(保障新人前九個月每月5萬元!)_A H62', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
{'header': {'jobName': '不動產儲備經理人(保障新人前九個月每月5萬元!)_B H62', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
{'header': {'jobName': '不動產儲備經理人(保障新人前九個月每月5萬元!)_C H62', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
{'header': {'jobName': '不動產儲備經理人(保障新人前九個月每月5萬元!)_D E63', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
{'header': {'jobName': '不動產儲備經理人(保障新人前九個月每月5萬元!)_E E63', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
{'header': {'jobName': '不動產儲備經理人(保障新人前九個月每月5萬元!)_F E63', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'cus

75700
75800
75900
76000
76100
76200
76300
76400
76500
76600
76700
76800
76900
77000
77100
77200
77300
77400
77500
77600
77700
77800
77900
78000
78100
78200
78300
78400
78500
78600
{'header': {'jobName': '送貨司機(台中)', 'custName': '晧晟實業股份有限公司'}, 'switch': 'off', 'custNo': '23075143000'}
78700
78800
78900
79000
79100
79200
79300
79400
79500
79600
79700
79800
{'header': {'jobName': '工程主管(台南/高雄)', 'custName': '實威國際股份有限公司'}, 'switch': 'off', 'custNo': '27746072000'}
79900
80000
80100
80200
80300
80400
80500
80600
80700
80800
80900
81000
81100
81200
81300
81400
81500
81600
81700
81800
81900
82000
82100
82200
82300
82400
82500
82600
82700
82800
82900
83000
83100
83200
83300
83400
83500
83600
83700
83800
83900
84000
84100
84200
84300
84400
84500
84600
84700
84800
84900
85000
{'header': {'jobName': '早班正職人員', 'custName': '統一超商秋紅谷門市_佳昕商行'}, 'switch': 'off', 'custNo': '130000000128115'}
85100
85200
85300
85400
85500
85600
85700
85800
85900
86000
86100
86200
86300
86400
86500
86600
86700
86800
86900
8

96800
96900
97000
97100
97200
97300
97400
97500
97600
97700
97800
97900
98000
98100
98200
98300
98400
98500
98600
98700
98800
98900
99000
99100
99200
99300
99400
99500
99600
99700
99800
99900
100000
100100
100200
100300
100400
100500
100600
100700
100800
100900
101000
101100
101200
101300
101400
101500
101600
101700
101800
101900
102000
102100
102200
102300
102400
{'header': {'jobName': '行銷業務', 'custName': '歐立吉諾廣告行銷有限公司'}, 'switch': 'off', 'custNo': '12994242000'}
102500
102600
102700
102800
102900
103000
103100
103200
103300
103400
103500
103600
103700
103800
103900
104000
104100
104200
104300
104400
104500
104600
104700
104800
104900
105000
{'header': {'jobName': '廚房假日工讀生（中科店）', 'custName': '聚鼎盛國際餐飲有限公司'}, 'switch': 'off', 'custNo': '130000000138190'}
{'header': {'jobName': '外場正職（中科店）', 'custName': '聚鼎盛國際餐飲有限公司'}, 'switch': 'off', 'custNo': '130000000138190'}
{'header': {'jobName': '外場服務人員', 'custName': '貓門_貓門事業有限公司'}, 'switch': 'off', 'custNo': '130000000138287'}
105100
105200
10530

162500
162600
162700
162800
162900
163000
163100
163200
163300
163400
163500
163600
163700
163800
163900
164000
164100
164200
{'header': {'jobName': '[發財燒臘青海店]內場廚房人員(有炒台砧板經驗佳)', 'custName': '香港發財燒腊店_發財餐飲有限公司'}, 'switch': 'off', 'custNo': '130000000070247'}
164300
164400
164500
164600
164700
164800
164900
165000
165100
165200
165300
165400
165500
165600
165700
165800
165900
166000
166100
166200
166300
166400


In [ ]:
df.to_pickle('./data/JobDetail.pkl')

In [8]:
df

,jobName,appearDate,custName,hrName,email,phone,acceptRole,workExp,edu,major,...,workType,addressRegion,addressDetail,manageResp,businessTrip,workPeriod,vacationPolicy,startWorkingDay,hireType,industry
0,作業員,2020/09/15,總信食品有限公司,胡慧敏小姐,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",不拘,高中、專科、大學,[],...,[],台中市南屯區,工業區二十二路17號,不需負擔管理責任,無需出差外派,日班/晚班，需輪班,週休二日,不限,0,食品什貨批發業
1,汐止分院護理人員(內外科病房.加護病房.門診.安寧病房.護理之家),2020/09/15,國泰醫療財團法人國泰綜合醫院,李宿華小姐,sue@cgh.org.tw,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",不拘,專科、大學,[護理助產相關],...,[],新北市汐止區,建成路59巷2號,不需負擔管理責任,無需出差外派,日班/晚班，需輪班,依公司規定,不限,0,醫院
2,印刷見習技師,2020/09/18,康揚特殊美術印刷股份有限公司,邱先生,ford521213@yahoo.com.tw,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",不拘,高中、專科、大學,[],...,[],台北市南港區,南港路三段50巷9號5樓,不需負擔管理責任,無需出差外派,日班/晚班，08:00~17:30，需輪班,週休二日,一週內,0,印刷業
3,證券營業員(南科分公司),2020/09/15,國票綜合證券股份有限公司,郭經理,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",不拘,專科以上,[],...,[],台南市新市區,',不需負擔管理責任,無需出差外派,日班,依公司規定,不限,0,證券及期貨業
4,美編設計(需具經驗),2020/09/19,益盛多媒體印刷有限公司,游先生,yisheng.4910@gmail.com,02-22224910-32,"[{'code': 1, 'description': '上班族'}]",3年以上,不拘,[],...,[],新北市中和區,立德街98巷123號2樓,不需負擔管理責任,無需出差外派,日班，例如：8:30~17:30,週休二日,一週內,0,印刷業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200724,"優質人才培訓專案人員(承攬性質,非勞動契約)",2020/09/01,南山人壽保險股份有限公司_中壢佰億通訊處(林美玲),林美玲處經理,g22558@ms21.hinet.net,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",不拘,高中以上,[],...,[],桃園市平鎮區,環南路50號4樓,不需負擔管理責任,無需出差外派,日班，(依公司規定),依公司規定,不限,0,人身保險業
200725,南部寵物商品業務人員(高嘉南區) (底薪+獎金),2020/09/16,聯享寵物貿易有限公司,林小姐,,07-5566280,"[{'code': 1, 'description': '上班族'}]",3年以上,高中以上,[],...,[],高雄市左營區,博愛二路198號11樓之1,不需負擔管理責任,無需出差外派,日班,週休二日,兩週內,0,寵物相關服務業
200726,準公幼貝萊登幼兒園-高年級安親班老師,2020/09/07,嘉義市私立貝萊登幼兒園,李小姐,,05-2752988,"[{'code': 1, 'description': '上班族'}]",1年以上,大學以上,"[學前教育相關, 綜合教育相關, 專業科目教育相關]",...,[],嘉義市,東區林森東路539號,不需負擔管理責任,無需出差外派,日班/晚班,依公司規定,一個月內,0,學前教育事業
200727,Java程式開發工程師,2020/09/14,藍科數位科技股份有限公司,陳小姐,,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",1年以上,專科以上,[],...,[],台北市內湖區,行愛路151號2樓之2,不需負擔管理責任,需外派，一年累積時間未定,日班，09:00~18:00,週休二日,不限,0,電腦軟體服務業


In [17]:
df['major'].value_counts()[:20]

[]                     136487
[電機電子工程相關]               2765
[護理助產相關]                 1952
[商業及管理學科類]               1459
[機械工程相關]                 1252
[電機電子工程相關, 資訊工程相關]       1205
[資訊管理相關, 資訊工程相關]         1064
[資訊工程相關, 資訊管理相關]         1040
[工程學科類]                   963
[醫學系相關]                   831
[會計學相關]                   756
[電機電子工程相關, 機械工程相關]        747
[資訊工程相關]                  718
[土木工程相關, 建築相關]            677
[機械工程相關, 電機電子工程相關]        617
[資訊工程相關, 電機電子工程相關]        561
[會計學相關, 財稅金融相關]           509
[藥學相關]                    490
[建築相關, 土木工程相關]            465
[工程學科類, 工業技藝及機械學科類]       364
Name: major, dtype: int64